# Demand side flexibility


In [1]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using CSV, DataFrames, JuMP, Gurobi

# Cost of the technologies

In [3]:
#Fixed cost of plant (per power [$/MW])
C_FC = [
    # Solar
    # Wind
    # CCGT
    # Coal
    # Battery storage in Power
    # Battery storage in energy
] #column vector of size I (number of mean of production)

# Fixed operation and maintenance cost (per power [$/MW])
C_FOM = [
    # Solar
    # Wind
    # CCGT
    # Coal
    # Battery storage in Power
    # Battery storage in energy (might be equal to zero)
]#column vector of size I (number of mean of production)

# Variable operation and maintenance cost (per energy [$/MWh])
C_VOM = [
    # Solar (0)
    # Wind (0)
    # CCGT (Fuel Price ?)
    # Coal (Fuel Price?)
    # Battery storage in Power (0)
    # Battery storage in energy (0)
] #column vector of size I (number of mean of production)

#Carbon intensity (tons of CO2 per MWh [ton/MWh])
E_CO2 = = [
    # Solar (0)
    # Wind (0)
    # CCGT (to be find)
    # Coal (to be find)
    # Battery storage in Power (0)
    # Battery storage in energy (0)
] # column vector of size I (number of mean of production)

# Demand data

In [ ]:
# Demand vector
X_D = [
    X_D_t1, # Energy demand at hour 1 in MW or in MWh (would be the same as we used a one hour time frame)
    X_D_t2, 
    ...

]

# We would have to derive this vector with and without demand side flexibility

# Availability for RE

In [ ]:
Availability =ones(lenght(renewable_ls), lenght(hr_in_year))

# Availability[i, t] availability of the renewable i at the time t

# i =1 => solar
# i = 2 => wind

# Constant

In [ ]:
eta = # Efficiency of the battery

CO2_price = # price of the CO2 per ton

# Storage model

In [ ]:
function storage_charge(x_g, x_d, x_soc, P_s, h, t)
    # Charge of the battery

    if x_g[t] > x_d[t]
        # If the generation is above the demand we can charge
        x_ch[t] = minimum([eta * (x_g[t] - x_d[t]), eta * P_s * 1, (1 - x_soc[t])*h*P_s])
        # We charge up to : the energy available above the demand, or the maximum battery capacity, or 
        # the available energy storage left in the battery
    else
        # If we don't have excess energy we don't charge
        x_ch[t] = 0
    end
    return x_ch[t]
end

function storage_discharge(x_g, x_d, x_soc, P_s, h, t)
        # Discharge of the battery
        if x_g[t] < x_d[t]
            # We discharge if the generation from the other sources is not enought
            x_dch[t] = minimum([(x_g[t] - x_d[t]), P_s * 1 , x_soc_t*h*P_s])
        
        else
            # If we already have enough energy we don't discharge
            x_dch[t] = 0
    
        end
    return x_dch[t]
end

function storage_state_of_charge(x_g, x_d, x_soc, P_s, h, t)
    # Computation of the state of charge 
    if t == 1
        x_soc[t] = 1 
    
    else
        # State of charge can be computed from the previous state at t-1
        x_soc[t] = x_soc[t-1] + (x_ch[t-1] - x_dch[t-1])/(h*P_s)
    end
        
return x_soc[t]
end


# Model Functions

In [ ]:
function generation_non_storage(P, X_ff ,Availability, t)

    # P[1] : Solar 
    #Either we generate at the max capacity or at the max availability
    X_G_non_storage[1] = minimum([P[1] * 1 , Availability[1, t] * 1])

    # P[2] : Wind 
    #Either we generate at the max capacity or at the max availability
    X_G_non_storage[2] = minimum([P[2] * 1 , Availability[2, t] * 1])

    # P[3] : CCGT 
    X_G_non_storage[3] = minimum([P[3] * 1 , X_ff[1, t]])

    # P[4] : Coal 
    X_G_non_storage[3] = minimum([P[4] * 1 , X_ff[2, t]])

    x_g[t] = sum(X_G_non_storage)
end

function cost_mix(P, X)
    # Function that compute the cost of the mix from all the cost
    cost = sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)) + sum(sum((C_VOM[i] + CO2_price * E_CO2)* X[i,t] for i in 1:size(X,1)) for t in 1:size(X,2)) 

# Creating the model

List of variable 

In [ ]:
Power_generation_ls = 1:6 # Solar, Wind, CCGT, Coal, Battery Power, Battery time

fossil_fuel_ls = 1:2 # CCGT, Coal

In [17]:
model = Model(Gurobi.Optimizer)

@variable(model, P[Power_generation_ls] >= 0)

@variable(model, X_ff[fossil_fuel_ls] >= 0)

@objective(model, Min, sum(sum(d[i,j]* X[i,j] for j in landfills_ls) for i in centers_ls))

@constraint(model, waste_manegement[i in centers_ls], sum(X[i,j] for j in landfills_ls)  == q[i])

@constraint(model, [i = centers_ls, j = landfills_ls], X[i,j]   <= 1000000 * Z[j])

@constraint(model, landfills_construnction, sum(Z[j] for j in landfills_ls)   <= 5);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07


Optimize



In [18]:
optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 801 rows, 765 columns and 2265 nonzeros
Model fingerprint: 0x0305a04b
Variable types: 750 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+03]
Presolve time: 0.01s
Presolved: 801 rows, 765 columns, 2265 nonzeros
Variable types: 750 continuous, 15 integer (15 binary)
Found heuristic solution: objective 1026891.5990

Root relaxation: objective 8.404875e+05, 136 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    840487.52400 840487.524  0.00%     -    0s

Explored 1 nodes (136 simplex iterations) i

In [19]:
opti_cost = objective_value(model)

println(opti_cost)


840487.5240012797


In [21]:
X_matrix = ones(size(X))


for i in 1:size(X,1)
    for j in 1:size(X,2)
        X_matrix[i,j] = value(X[i,j])
    end
end
println(size(X_matrix))


Z_matrix = ones(size(Z))


for i in 1:size(Z,1)

    Z_matrix[i] = value(Z[i])

end

println(Z_matrix)

(50, 15)
[-0.0, 0.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, 0.0, 0.0, 1.0, 1.0, -0.0]


Number of customers served fully from the closest things

In [22]:
cust_closest = []
nb_cst_closest = 0

for i = customers_ls
    if X_matrix[Int(ind_min[i]),i] >= customers[i,3]
        cust_closest = [cust_closest; i]
        nb_cst_closest = nb_cst_closest + 1
    end
end

println(cust_closest)
println(nb_cst_closest)

Any[2, 4, 9, 12, 13, 14, 17, 19, 20, 21, 22, 24, 28, 29, 31, 35, 36, 38, 43, 45, 46, 47, 51, 53, 54, 58, 59, 62, 64, 65, 66, 69, 70, 77, 78, 81, 82, 84, 88, 89, 92, 97, 98, 99, 101, 103, 104, 108, 112, 116, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 130, 137, 138, 139, 140, 142, 143, 144, 145, 146, 153, 154, 157, 158, 160, 165, 167, 170, 171, 173, 175, 176, 177, 178, 179, 180, 182, 183, 184, 185, 187, 188, 190, 191, 195, 198, 201, 202, 204, 206, 207, 208, 210, 215, 216, 220, 224, 228, 229, 233, 236, 238, 239, 242, 243, 244, 246, 250, 259, 264, 265, 269, 270, 272, 275, 276, 279, 280, 283, 285, 287, 288, 290, 292, 305, 307, 308, 311, 313, 317, 318, 321, 322, 323, 326, 327, 328, 329, 334, 336, 337, 339, 340, 341, 342, 345, 346, 348, 351, 352, 357, 358, 360, 361, 362, 367, 368, 370, 371, 376, 377, 378, 379, 380, 381, 382, 384, 386, 393, 395, 399, 406, 411, 413, 416, 420, 422, 423, 424, 425, 429, 434, 435, 440, 444, 445, 451, 452, 453, 454, 455, 459, 460, 465, 468, 469, 471, 474, 477